<a href="https://colab.research.google.com/github/JeanMichelBereder/AtCoder/blob/main/haisyagashira_create_geotargeting_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 概要**
配車頭プロモーションのためにジオターゲティング広告が有効と思われる。

そのためには住所リストが必要となる。

住所データのデータソースは以下2つとなる。



*   [環境省産業廃棄物処理業者情報検索システム](https://www.env.go.jp/recycle/waste/sanpai/index.php)
*   [産廃情報ネット](https://www2.sanpainet.or.jp/)



# **2. クローラーの実装**

## **2.0 準備**
産廃ネットのクローリングではブラウザ操作が必要になるため、`selenium`と`chromewebdriver`をインストールしておく。

In [1]:
# 日本語フォントインストール
!apt install fonts-ipafont-gothic

# Seleniumインストール
!pip install selenium

# chromedriverインストール
!apt update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-ipafont-gothic is already the newest version (00303-18ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 77 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learni

必要なライブラリをここで読み込んでおく。

In [2]:
# Webスクレイピングに用いるためのライブラリ
from bs4 import BeautifulSoup
from urllib import request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import html

# プログレスバー表示用ライブラリ
import tqdm

# データ操作のためのライブラリ
import pandas as pd

# データ整形のためのライブラリ
from pprint import pprint
from datetime import datetime as dt
from datetime import timezone
from datetime import timedelta

# データをローカル保存するためのライブラリ
from google.colab import files

## **2.1 環境省検索システムのクローラー**

In [3]:
def get_data_from_kankyosho_HP(max_page_num: int) -> pd.DataFrame:
    """
    環境省のHPをクローリングするための関数。
    max_page_numを指定すると、そのページまでをクローリングしてくる。
    返り値はpandas DataFrame形式。
    """
    # URLの基本形式。p=の後にページ番号の数字が入る。
    base_url = 'https://www.env.go.jp/recycle/waste/sanpai/search_list.php?p='
    
    # 1ページごとに作ったDataFrameを格納しておくための配列
    df_list = []

    # 指定したmax_page_numまでイテレーションを回す
    for page_num in tqdm.tqdm(range(max_page_num)):
        # range関数は0からスタートなので、1足した上でURLに結合させる。
        url = base_url + str(page_num+1)
        # print('[%d] URL : %s' % (page_num, url))

        # BeautifulSoupのインスタンスを作る
        response = request.urlopen(url)
        soup = BeautifulSoup(response)
        response.close()

        # 業者一覧をパースした情報を格納しておくリスト
        result_list = []

        # 業者一覧の情報をパースしてresult_list配列に格納
        for line in soup.find('div',id='result').find_all('tr'):
            result_list.append(line.text.strip().split('\n'))

        # result_listに入っているデータをpandasに変換。0番目のデータはカラム名に相当する
        df = pd.DataFrame(result_list[1:],columns=result_list[0])
        df_list.append(df)
    
    # 業者データを格納済みのdataframeを結合する
    final_df = pd.concat(df_list)

    return final_df

In [4]:
# 最大ページ数を指定してクロールを実行
df_kankyosho = get_data_from_kankyosho_HP(max_page_num=5)

100%|██████████| 5/5 [00:01<00:00,  3.11it/s]


In [5]:
df_kankyosho

,業者名,所在地,固有番号,許可番号,許可主体,業の区分,許可年月日
0,道東荷役有限会社,北海道野付郡別海町別海２２９−１６,069094,00100069094,北海道,収集運搬,2020（平成32）年5月22日
1,株式会社上田組,北海道標津郡標津町字川北９６−５,072053,00100072053,北海道,収集運搬,2020（平成32）年10月23日
2,有限会社尾田運輸,北海道目梨郡羅臼町礼文町１５,072707,00100072707,北海道,収集運搬,2020（平成32）年10月13日
3,株式会社大樹貨物,北海道広尾郡大樹町字下大樹１８５−３,075977,00100075977,北海道,収集運搬,2021（平成33）年4月2日
4,環境サービス有限会社,北海道標津郡標津町南４条東１−１−１０,076141,00100076141,北海道,収集運搬,2021（平成33）年4月17日
...,...,...,...,...,...,...,...
15,三央工業株式会社,北海道札幌市白石区北郷８条８−２−８,004827,00100004827,北海道,収集運搬,2018（平成30）年7月14日
16,札幌第一清掃株式会社,北海道札幌市西区二十四軒四条２−２−１,004828,00100004828,北海道,収集運搬,2018（平成30）年7月29日
17,大興運輸倉庫株式会社,東京都江東区枝川２−７−８,004830,00100004830,北海道,収集運搬,2018（平成30）年8月11日
18,株式会社とませい,北海道苫小牧市新開町２−２−１０,004831,00110004831,北海道,収集運搬,2015（平成27）年8月31日


In [6]:
# ファイル名にタイムスタンプを付与する
timestamp = dt.now(timezone(timedelta(hours=9))).strftime('_%Y%m%d_%H%M%S')
filename = "産業廃棄物業者一覧%s.csv" % (timestamp)

# CSVファイルとしてダウンロード
df_kankyosho.reset_index(drop=True).to_csv(filename, encoding='utf-8')
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **2.2 産廃情報ネットのクローラー**


In [7]:
def get_data_from_sampainet_HP(get_all: bool) -> pd.DataFrame:
    """
    産廃ネットのHPをクローリングするための関数。
    返り値はpandas DataFrame形式。
    """
    # アクセスするURL（優良認定業者検索ページ）
    url = 'https://www2.sanpainet.or.jp/index_u4.php'
    
    # seleniumによってwebブラウザーを自動操作
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.implicitly_wait(10)
    driver.get(url)

    # frameを移動
    frame = driver.find_element(by=By.XPATH, value='/html/frameset/frame[2]')
    driver.switch_to.frame(frame)

    # pagesourceを表示
    # print(driver.page_source)

    # 優良認定を受けた業の区分にある4つの条件をすべて選び、クリックする
    param1 = driver.find_element(by=By.XPATH, value='/html/body/form[1]/table[1]/tbody/tr[3]/td/table[1]/tbody/tr[4]/td/table/tbody/tr/td[1]/input[1]')
    param2 = driver.find_element(by=By.XPATH, value='/html/body/form[1]/table[1]/tbody/tr[3]/td/table[1]/tbody/tr[4]/td/table/tbody/tr/td[2]/input[1]')
    param3 = driver.find_element(by=By.XPATH, value='/html/body/form[1]/table[1]/tbody/tr[3]/td/table[1]/tbody/tr[4]/td/table/tbody/tr/td[1]/input[2]')
    param4 = driver.find_element(by=By.XPATH, value='/html/body/form[1]/table[1]/tbody/tr[3]/td/table[1]/tbody/tr[4]/td/table/tbody/tr/td[2]/input[2]')
    param1.click()
    param2.click()
    param3.click()
    param4.click()
    print('Param button clicked ... ')

    # 検索ボタンをクリック
    search_button = driver.find_element(by=By.XPATH, value='/html/body/form[1]/table[2]/tbody/tr/td/table/tbody/tr/td[1]/input')
    search_button.click()
    print('Search button clicked ... ')
    
    # 全部表示のボタンをクリック
    if get_all:
        all_button = driver.find_element(by=By.XPATH, value='/html/body/form/table[2]/tbody/tr[3]/td/table/tbody/tr/td[1]/a[3]')
        all_button.click()
        print('Select All button clicked ... ')

    # 事業所一覧のテーブルを取得
    table = driver.find_element(by=By.XPATH, value='/html/body/form/table[2]/tbody/tr[4]/td/table/tbody').find_elements(By.XPATH, value='.//tr')

    # 結果格納用のリスト
    column_name = []
    table_data = []
    column_name = ['許可主体', '業者名', '代表者', '所在地', '業の区分', '許可番号', '優良認定業者',  '許可年月日', '許可期限年月日']
    print('Start scraping...')
    for line_index, table_line in tqdm.tqdm(enumerate(table[1:])):
        table_data_line = []
        for column_index, table_line_value in enumerate(table_line.find_elements(by=By.XPATH, value='.//td')[:4]):
            tmp = table_line_value.text.replace('\n', '_')
            table_data_line.append(table_line_value.text.replace('\n', '_'))
        table_data.append([e for e in '_'.join(table_data_line).split('_')])
    

    # 返り値となる pandas DataFrame
    df = pd.DataFrame(table_data, columns=column_name)
    print('Succeed')

    return df

In [8]:
# クロールを実行（get_all=Trueにすると全情報取得する。1時間半くらいかかる）
df_sanpainet = get_data_from_sampainet_HP(get_all=False)
df_sanpainet

Param button clicked ... 
Search button clicked ... 
Start scraping...


20it [00:03,  6.20it/s]

Succeed


,許可主体,業者名,代表者,所在地,業の区分,許可番号,優良認定業者,許可年月日,許可期限年月日
0,北海道,早来工営株式会社,代表取締役 小松 稔明,神奈川県川崎市川崎区扇町６－１,産業廃棄物収集運搬業,第00110001169号,「優良認定業者」,2015(平成27)年03月12日,2022(令和4)年02月21日
1,北海道,角山開発株式会社,代表取締役 湯藤 学,北海道江別市角山６９－４,特別管理産業廃棄物収集運搬業,第00150005112号,「優良認定業者」,2015(平成27)年03月03日,2022(令和4)年03月02日
2,北海道,静和エンバイロメント株式会社,代表取締役 和波 剛,静岡県静岡市駿河区豊田２－７－９,産業廃棄物収集運搬業,第00100051046号,「優良認定業者」,2015(平成27)年07月08日,2022(令和4)年06月24日
3,北海道,静和エンバイロメント株式会社,代表取締役 和波 剛,静岡県静岡市駿河区豊田２－７－９,特別管理産業廃棄物収集運搬業,第00150051046号,「優良認定業者」,2015(平成27)年07月08日,2022(令和4)年06月24日
4,北海道,北清企業株式会社,代表取締役 川井 雄一,北海道札幌市東区北丘珠５条４－５－７,特別管理産業廃棄物収集運搬業,第00150004495号,「優良認定業者」,2016(平成28)年02月04日,2022(令和4)年06月30日
5,北海道,アサヒプリテック株式会社,代表取締役 中西 広幸,兵庫県神戸市東灘区魚崎浜町２１,特別管理産業廃棄物処分業,第00170000503号,「優良認定業者」,2015(平成27)年07月18日,2022(令和4)年07月12日
6,北海道,サンワリューツー株式会社,代表取締役 谷口 隆司,愛知県刈谷市一里山町家下８０,特別管理産業廃棄物収集運搬業,第00150005459号,「優良認定業者」,2015(平成27)年07月27日,2022(令和4)年07月26日
7,北海道,北清企業株式会社,代表取締役 川井 雄一,北海道札幌市東区北丘珠５条４－５－７,産業廃棄物収集運搬業,第00100004495号,「優良認定業者」,2015(平成27)年08月05日,2022(令和4)年08月04日
8,北海道,ミヤマ株式会社,代表取締役 南 克明,長野県長野市稲里１－５－３,産業廃棄物収集運搬業,第00100000553号,「優良認定業者」,2015(平成27)年09月16日,2022(令和4)年08月29日
9,北海道,株式会社とませい,代表取締役 渡辺 健治,北海道苫小牧市新開町２－２－１０,産業廃棄物収集運搬業,第00110004831号,「優良認定業者」,2015(平成27)年08月31日,2022(令和4)年08月30日


In [9]:
# ファイル名にタイムスタンプを付与する
timestamp = dt.now(timezone(timedelta(hours=9))).strftime('_%Y%m%d_%H%M%S')
filename = "優良産業廃棄物業者一覧%s.csv" % (timestamp)

# CSVファイルとしてダウンロード
df_sanpainet.reset_index(drop=True).to_csv(filename, encoding='utf-8')
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **3. データの整形/結合**

上記で取得したデータを整形・結合する。

In [17]:
# CSVをアップロード
files.upload()

Saving 産業廃棄物業者一覧_20220602_140651.csv to 産業廃棄物業者一覧_20220602_140651.csv
Saving 優良産業廃棄物業者一覧_20220602_135140.csv to 優良産業廃棄物業者一覧_20220602_135140.csv


In [18]:
# CSVを読み込んでpandasにする
df_kankyosho = pd.read_csv('/content/産業廃棄物業者一覧_20220602_140651.csv', index_col=0)
df_sanpainet = pd.read_csv('/content/優良産業廃棄物業者一覧_20220602_135140.csv', index_col=0)

In [19]:
# 環境省HPのデータ表示
df_kankyosho

,業者名,所在地,固有番号,許可番号,許可主体,業の区分,許可年月日
0,道東荷役有限会社,北海道野付郡別海町別海２２９−１６,69094,100069094,北海道,収集運搬,2020（平成32）年5月22日
1,株式会社上田組,北海道標津郡標津町字川北９６−５,72053,100072053,北海道,収集運搬,2020（平成32）年10月23日
2,有限会社尾田運輸,北海道目梨郡羅臼町礼文町１５,72707,100072707,北海道,収集運搬,2020（平成32）年10月13日
3,株式会社大樹貨物,北海道広尾郡大樹町字下大樹１８５−３,75977,100075977,北海道,収集運搬,2021（平成33）年4月2日
4,環境サービス有限会社,北海道標津郡標津町南４条東１−１−１０,76141,100076141,北海道,収集運搬,2021（平成33）年4月17日
...,...,...,...,...,...,...,...
46775,南舘 正彦,宮城県白石市字沢目１３６−３,61401,1000061401,群馬県,収集運搬,2018（平成30）年7月23日
46776,みなと運送株式会社,茨城県神栖市知手中央１０−７−４３,576,1000000576,群馬県,収集運搬,2020（平成32）年5月8日
46777,株式会社ハッポーライフ彩生,埼玉県久喜市河原井町４７−１,70065,1000070065,群馬県,収集運搬,2020（平成32）年10月27日
46778,天童環境株式会社,山形県天童市石鳥居２−２−５０,9555,1000009555,群馬県,収集運搬,2020（平成32）年6月22日


In [21]:
# 環境省データの許可番号を10桁にゼロ埋めする
df_kankyosho['許可番号'] = df_kankyosho['許可番号'].astype(str).str.zfill(10)

In [22]:
df_kankyosho

,業者名,所在地,固有番号,許可番号,許可主体,業の区分,許可年月日
0,道東荷役有限会社,北海道野付郡別海町別海２２９−１６,69094,0100069094,北海道,収集運搬,2020（平成32）年5月22日
1,株式会社上田組,北海道標津郡標津町字川北９６−５,72053,0100072053,北海道,収集運搬,2020（平成32）年10月23日
2,有限会社尾田運輸,北海道目梨郡羅臼町礼文町１５,72707,0100072707,北海道,収集運搬,2020（平成32）年10月13日
3,株式会社大樹貨物,北海道広尾郡大樹町字下大樹１８５−３,75977,0100075977,北海道,収集運搬,2021（平成33）年4月2日
4,環境サービス有限会社,北海道標津郡標津町南４条東１−１−１０,76141,0100076141,北海道,収集運搬,2021（平成33）年4月17日
...,...,...,...,...,...,...,...
46775,南舘 正彦,宮城県白石市字沢目１３６−３,61401,1000061401,群馬県,収集運搬,2018（平成30）年7月23日
46776,みなと運送株式会社,茨城県神栖市知手中央１０−７−４３,576,1000000576,群馬県,収集運搬,2020（平成32）年5月8日
46777,株式会社ハッポーライフ彩生,埼玉県久喜市河原井町４７−１,70065,1000070065,群馬県,収集運搬,2020（平成32）年10月27日
46778,天童環境株式会社,山形県天童市石鳥居２−２−５０,9555,1000009555,群馬県,収集運搬,2020（平成32）年6月22日


In [20]:
# 産廃ネットの情報表示
df_sanpainet

,許可主体,業者名,代表者,所在地,業の区分,許可番号,優良認定業者,許可年月日,許可期限年月日
0,北海道,早来工営株式会社,代表取締役 小松 稔明,神奈川県川崎市川崎区扇町６－１,産業廃棄物収集運搬業,第00110001169号,「優良認定業者」,2015(平成27)年03月12日,2022(令和4)年02月21日
1,北海道,角山開発株式会社,代表取締役 湯藤 学,北海道江別市角山６９－４,特別管理産業廃棄物収集運搬業,第00150005112号,「優良認定業者」,2015(平成27)年03月03日,2022(令和4)年03月02日
2,北海道,静和エンバイロメント株式会社,代表取締役 和波 剛,静岡県静岡市駿河区豊田２－７－９,産業廃棄物収集運搬業,第00100051046号,「優良認定業者」,2015(平成27)年07月08日,2022(令和4)年06月24日
3,北海道,静和エンバイロメント株式会社,代表取締役 和波 剛,静岡県静岡市駿河区豊田２－７－９,特別管理産業廃棄物収集運搬業,第00150051046号,「優良認定業者」,2015(平成27)年07月08日,2022(令和4)年06月24日
4,北海道,北清企業株式会社,代表取締役 川井 雄一,北海道札幌市東区北丘珠５条４－５－７,特別管理産業廃棄物収集運搬業,第00150004495号,「優良認定業者」,2016(平成28)年02月04日,2022(令和4)年06月30日
...,...,...,...,...,...,...,...,...,...
12652,沖縄県,九州バイオカーボン株式会社,代表取締役 新名 洋一郎,福岡県太宰府市大字北谷字夕内１１２５－２,産業廃棄物処分業,第04724008037号,「優良認定業者」,2021(令和3)年12月01日,2028(令和10)年11月30日
12653,沖縄県,株式会社勝利商會,代表取締役 下田 勝利,鹿児島県鹿児島市小川町２７－２,特別管理産業廃棄物収集運搬業,第04754002945号,「優良認定業者」,2022(令和4)年03月26日,2029(令和11)年03月25日
12654,沖縄県,株式会社勝利商會,代表取締役 下田 勝利,鹿児島県鹿児島市小川町２７－２,産業廃棄物収集運搬業,第04704002945号,「優良認定業者」,2022(令和4)年03月26日,2029(令和11)年03月25日
12655,沖縄県,株式会社環境開発公社,代表取締役 栗本 貴志,広島県広島市西区南観音６−１２−２１,産業廃棄物処分業,第04722007362号,「優良認定業者」,2022(令和4)年04月04日,2029(令和11)年04月03日


In [24]:
# 産廃ネットの許可番号を整形する
df_sanpainet['許可番号'] = df_sanpainet['許可番号'].str[1:-1].str.zfill(10)

In [25]:
df_sanpainet

,許可主体,業者名,代表者,所在地,業の区分,許可番号,優良認定業者,許可年月日,許可期限年月日
0,北海道,早来工営株式会社,代表取締役 小松 稔明,神奈川県川崎市川崎区扇町６－１,産業廃棄物収集運搬業,00110001169,「優良認定業者」,2015(平成27)年03月12日,2022(令和4)年02月21日
1,北海道,角山開発株式会社,代表取締役 湯藤 学,北海道江別市角山６９－４,特別管理産業廃棄物収集運搬業,00150005112,「優良認定業者」,2015(平成27)年03月03日,2022(令和4)年03月02日
2,北海道,静和エンバイロメント株式会社,代表取締役 和波 剛,静岡県静岡市駿河区豊田２－７－９,産業廃棄物収集運搬業,00100051046,「優良認定業者」,2015(平成27)年07月08日,2022(令和4)年06月24日
3,北海道,静和エンバイロメント株式会社,代表取締役 和波 剛,静岡県静岡市駿河区豊田２－７－９,特別管理産業廃棄物収集運搬業,00150051046,「優良認定業者」,2015(平成27)年07月08日,2022(令和4)年06月24日
4,北海道,北清企業株式会社,代表取締役 川井 雄一,北海道札幌市東区北丘珠５条４－５－７,特別管理産業廃棄物収集運搬業,00150004495,「優良認定業者」,2016(平成28)年02月04日,2022(令和4)年06月30日
...,...,...,...,...,...,...,...,...,...
12652,沖縄県,九州バイオカーボン株式会社,代表取締役 新名 洋一郎,福岡県太宰府市大字北谷字夕内１１２５－２,産業廃棄物処分業,04724008037,「優良認定業者」,2021(令和3)年12月01日,2028(令和10)年11月30日
12653,沖縄県,株式会社勝利商會,代表取締役 下田 勝利,鹿児島県鹿児島市小川町２７－２,特別管理産業廃棄物収集運搬業,04754002945,「優良認定業者」,2022(令和4)年03月26日,2029(令和11)年03月25日
12654,沖縄県,株式会社勝利商會,代表取締役 下田 勝利,鹿児島県鹿児島市小川町２７－２,産業廃棄物収集運搬業,04704002945,「優良認定業者」,2022(令和4)年03月26日,2029(令和11)年03月25日
12655,沖縄県,株式会社環境開発公社,代表取締役 栗本 貴志,広島県広島市西区南観音６−１２−２１,産業廃棄物処分業,04722007362,「優良認定業者」,2022(令和4)年04月04日,2029(令和11)年04月03日


In [26]:
# データを結合
df_merged = pd.merge(df_kankyosho, df_sanpainet, on='許可番号')
df_merged

,業者名_x,所在地_x,固有番号,許可番号,許可主体_x,業の区分_x,許可年月日_x,許可主体_y,業者名_y,代表者,所在地_y,業の区分_y,優良認定業者,許可年月日_y,許可期限年月日
0,株式会社シヨーモン,埼玉県さいたま市見沼区大字片柳１０４５−１,6070,10110006070,さいたま市,収集運搬,2017（平成29）年7月20日,さいたま市,株式会社シヨーモン,代表取締役 松澤 敏也,埼玉県さいたま市見沼区大字片柳１０４５－１,産業廃棄物収集運搬業,「優良認定業者」,2017(平成29)年07月20日,2024(令和6)年06月07日
1,リバー株式会社,東京都墨田区緑１−４−１９,34488,10120034488,さいたま市,処分,2021（平成33）年7月20日,さいたま市,株式会社鈴徳,代表取締役 猪鼻 秀希,東京都墨田区緑１－４－１９,産業廃棄物処分業,「優良認定業者」,2021(令和3)年07月20日,2028(令和10)年05月23日
2,株式会社エコ計画,埼玉県さいたま市浦和区仲町４−２−２０,4328,10120004328,さいたま市,処分,2021（平成33）年10月26日,さいたま市,株式会社エコ計画,代表取締役 井上 綱隆,埼玉県さいたま市浦和区仲町４－２－２０,産業廃棄物処分業,「優良認定業者」,2021(令和3)年10月26日,2028(令和10)年08月31日
3,株式会社エコ計画,埼玉県さいたま市浦和区仲町４−２−２０,4328,10110004328,さいたま市,収集運搬,2021（平成33）年10月26日,さいたま市,株式会社エコ計画,代表取締役 井上 綱隆,埼玉県さいたま市浦和区仲町４－２－２０,産業廃棄物収集運搬業,「優良認定業者」,2021(令和3)年10月26日,2028(令和10)年08月31日
4,有限会社河口商店,愛知県岡崎市伊賀町字７−１１７,2692,10510002692,岡崎市,収集運搬,2018（平成30）年5月11日,岡崎市,有限会社河口商店,代表取締役 河口 誠,愛知県岡崎市伊賀町字７−１１７,産業廃棄物収集運搬業,「優良認定業者」,2018(平成30)年05月11日,2025(令和7)年05月10日
